In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import wordcloud
import matplotlib.pyplot as plt

from nltk.corpus import stopwords, opinion_lexicon
from nltk.stem import WordNetLemmatizer

# from spellchecker import SpellChecker
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from tqdm import tqdm
tqdm.pandas()

import os
for dirname, _, filenames in os.walk('aalto-snlp-course-competition-2024'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

aalto-snlp-course-competition-2024/dev_2024.csv
aalto-snlp-course-competition-2024/LSTM-modify-preprocess.ipynb
aalto-snlp-course-competition-2024/dev_2024_test.csv
aalto-snlp-course-competition-2024/dev_2024_train.csv
aalto-snlp-course-competition-2024/train_2024.csv
aalto-snlp-course-competition-2024/sample_submission_2024.csv
aalto-snlp-course-competition-2024/test_2024.csv


In [2]:
import csv
df_train = pd.read_csv('./aalto-snlp-course-competition-2024/train_2024.csv',quoting=csv.QUOTE_NONE)
df_dev = pd.read_csv('./aalto-snlp-course-competition-2024/dev_2024.csv',quoting=csv.QUOTE_NONE)
df_test = pd.read_csv('./aalto-snlp-course-competition-2024/test_2024.csv',sep=',',encoding='utf-8',quoting=csv.QUOTE_NONE)

In [3]:
import re
import string
lemmatizer = WordNetLemmatizer()
POSITIVE_WORDS = set(opinion_lexicon.positive())
NEGATIVE_WORDS = set(opinion_lexicon.negative())
# spell = SpellChecker()
def clean_text(text, set_lower=True):
    '''Make text lowercase, remove text in square brackets,remove links, remove html tags,remove punctuation
    and remove numbers.'''
    new_text = text
    if set_lower:
        new_text = new_text.lower()
    # remove text in square brackets
    new_text = re.sub('\[.*?\]', '', new_text)
    # remove links
    new_text = re.sub('https?://\S+|www\.\S+', '', new_text)
    # remove html tags
    new_text = re.sub('<.*?>', '', new_text)
    # remove punctuation
    new_text = re.sub('[%s]' % re.escape(string.punctuation), '', new_text)

    # Reference : https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b
    emoji_pattern = re.compile("["
                            u"\U0001F600-\U0001F64F"  # emoticons
                            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                            u"\U0001F680-\U0001F6FF"  # transport & map symbols
                            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                            u"\U00002702-\U000027B0"
                            u"\U000024C2-\U0001F251"
                            "]+", flags=re.UNICODE)
    new_text = emoji_pattern.sub(r'', new_text)
    new_text = re.sub('\n', '', new_text)
    new_text = re.sub('\w*\d\w*', '', new_text)
    new_text = re.sub('\d*', '', new_text)

    stop_words = set(stopwords.words('english'))
    tokens = []
    for word in new_text.split():
        if word not in stop_words:
            tokens.append(lemmatizer.lemmatize(word))
    new_text = ' '.join(tokens)
    return new_text

def extract_emoticons(text):
    '''Extract the last token
    '''
    new_text = clean_text(text, set_lower=False)
    words = new_text.split()
    return words[-1] if words else ''

def count_all_capitalized(text):
    new_text = clean_text(text, set_lower=False)
    return len([word for word in new_text.split() if word.isupper()])

def count_positive(text):
    new_text = clean_text(text, set_lower=False)
    return len([word for word in new_text.split() if word in POSITIVE_WORDS])

def count_negative(text):
    new_text = clean_text(text, set_lower=False)
    return len([word for word in new_text.split() if word in NEGATIVE_WORDS])
        

In [4]:
df_train['text_clean'] = df_train['text'].progress_apply(clean_text)
df_dev['text_clean'] = df_dev['text'].progress_apply(clean_text)
df_test['text_clean'] = df_test['text'].progress_apply(clean_text)

# df_train['text_clean_stopwords'] = df_train['text'].apply(clean_text, remove_stopwords=False)

# df_train['emoticons'] = df_train['text'].apply(extract_emoticons)
# df_dev['emoticons'] = df_dev['text'].apply(extract_emoticons)
# df_test['emoticons'] = df_test['text'].apply(extract_emoticons)

# df_train['num_all_capitalized'] = df_train['text'].apply(count_all_capitalized)
# df_dev['num_all_capitalized'] = df_dev['text'].apply(count_all_capitalized)
# df_test['num_all_capitalized'] = df_test['text'].apply(count_all_capitalized)

# df_train['num_positive'] = df_train['text'].apply(count_positive)
# df_dev['num_positive'] = df_dev['text'].apply(count_positive)
# df_test['num_positive'] = df_test['text'].apply(count_positive)

# df_train['num_negative'] = df_train['text'].apply(count_negative)
# df_dev['num_negative'] = df_dev['text'].apply(count_negative)
# df_test['num_negative'] = df_test['text'].apply(count_negative)

# df_train['num_pos_neg_ratio'] = (df_train['num_positive'] + 1) / (df_train['num_negative'] + 1)
# df_dev['num_pos_neg_ratio'] = (df_dev['num_positive'] + 1) / (df_dev['num_negative'] + 1)
# df_test['num_pos_neg_ratio'] = (df_test['num_positive'] + 1) / (df_test['num_negative'] + 1)

  0%|          | 0/99000 [00:00<?, ?it/s]

100%|██████████| 12001/12001 [00:01<00:00, 6666.74it/s]


In [5]:
df_train

,id,text,label,text_clean
0,0,Except that Desmond played first base last nig...,0,except desmond played first base last night ta...
1,1,What i find funny is the loyalty and blindness...,0,find funny loyalty blindness english community...
2,2,Read the article not just the headline & you ...,0,read article headline find
3,3,Speaking of a horses backside is that where y...,1,speaking horse backside head
4,4,Michael Barone- gee are you dumb. No other wo...,1,michael barone gee dumb word needed
...,...,...,...,...
98995,98995,the libs could just pass a law that pulls them...,1,libs could pas law pull treaty easily exited ball
98996,98996,Really? How does this post in any way relate t...,0,really post way relate article article “we can...
98997,98997,Hey illegals if your reading this get the hel...,1,hey illegals reading get hell country
98998,98998,"Excellent description ""he playground bully ol...",1,excellent description playground bully old lit...


In [6]:
words = []
for text in df_train['text_clean']:
    words.extend(text.split())
    
words = set(words)

# words_with_stopwords = []
# for text in df_train['text_clean_stopwords']:
#     words_with_stopwords.extend(text.split())

# words_with_stopwords = set(words_with_stopwords)

In [7]:
len(words)

79679

In [8]:
df_train

,id,text,label,text_clean
0,0,Except that Desmond played first base last nig...,0,except desmond played first base last night ta...
1,1,What i find funny is the loyalty and blindness...,0,find funny loyalty blindness english community...
2,2,Read the article not just the headline & you ...,0,read article headline find
3,3,Speaking of a horses backside is that where y...,1,speaking horse backside head
4,4,Michael Barone- gee are you dumb. No other wo...,1,michael barone gee dumb word needed
...,...,...,...,...
98995,98995,the libs could just pass a law that pulls them...,1,libs could pas law pull treaty easily exited ball
98996,98996,Really? How does this post in any way relate t...,0,really post way relate article article “we can...
98997,98997,Hey illegals if your reading this get the hel...,1,hey illegals reading get hell country
98998,98998,"Excellent description ""he playground bully ol...",1,excellent description playground bully old lit...


## Define models

In [9]:
# from imblearn.over_sampling import SMOTE
# from imblearn.pipeline import Pipeline, make_pipeline

from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.ensemble import StackingClassifier, RandomForestClassifier, AdaBoostClassifier,GradientBoostingClassifier
from sklearn import svm
from sklearn.metrics import f1_score, confusion_matrix, classification_report
preprocessing_pipe = Pipeline([
                    ('bow', CountVectorizer()), 
                    ('tfid', TfidfTransformer()),
                    # ('lda', LatentDirichletAllocation(n_components=1000)),
                ])
# preprocessing_pipe = Pipeline([
#                     ('tfidf2vec', TfidfVectorizer(max_features=6000)),
# ])
x_train, y_train = df_train['text_clean'], df_train['label']
x_dev, y_dev = df_dev['text_clean'], df_dev['label']
# x_train, y_train = df_train[['num_all_capitalized', 'num_pos_neg_ratio']], df_train['label']
# x_dev, y_dev = df_dev[['num_all_capitalized', 'num_pos_neg_ratio']], df_dev['label']
# Fit the pipeline with the data
lr_pipe = make_pipeline(preprocessing_pipe, LogisticRegression())
nb_pipe = make_pipeline(preprocessing_pipe, MultinomialNB())
rf_pipe = make_pipeline(preprocessing_pipe, RandomForestClassifier())
xgb_pipe = make_pipeline(preprocessing_pipe, XGBClassifier())
ada_pipe = make_pipeline(preprocessing_pipe, AdaBoostClassifier())
grad_pipe = make_pipeline(preprocessing_pipe, GradientBoostingClassifier())
svm_pipe = make_pipeline(preprocessing_pipe, svm.SVC())
cat_pipe = make_pipeline(preprocessing_pipe, CatBoostClassifier())
lgb_pipe = make_pipeline(preprocessing_pipe, LGBMClassifier())

In [10]:
estimators = [
    # ("Logistic Regression", lr_pipe),
    ("Naive Bayes", nb_pipe),
    ("Random Forest", rf_pipe),
    ("XGBoost", xgb_pipe),
#     ("AdaBoost", ada_pipe),
#     ("GradientBoostingClassifier", grad_pipe),
    # ("SVM", svm_pipe),
    # ("CatBoost", cat_pipe),
    # ("LGBM", lgb_pipe)
#     ("SVC", svm_pipe)
]

stacking_classifier = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression(), n_jobs=4)
stacking_classifier

StackingClassifier(estimators=[('Naive Bayes',
                                Pipeline(steps=[('pipeline',
                                                 Pipeline(steps=[('bow',
                                                                  CountVectorizer()),
                                                                 ('tfid',
                                                                  TfidfTransformer())])),
                                                ('multinomialnb',
                                                 MultinomialNB())])),
                               ('Random Forest',
                                Pipeline(steps=[('pipeline',
                                                 Pipeline(steps=[('bow',
                                                                  CountVectorizer()),
                                                                 ('tfid',
                                                                  TfidfTransformer())])),
                                                ('randomforestclassifier',
                                                 RandomForestClassifier()...
                                                               interaction_constraints=None,
                                                               learning_rate=None,
                                                               max_bin=None,
                                                               max_cat_threshold=None,
                                                               max_cat_to_onehot=None,
                                                               max_delta_step=None,
                                                               max_depth=None,
                                                               max_leaves=None,
                                                               min_child_weight=None,
                                                               missing=nan,
                                                               monotone_constraints=None,
                                                               n_estimators=100,
                                                               n_jobs=None,
                                                               num_parallel_tree=None,
                                                               predictor=None,
                                                               random_state=None, ...))]))],
                   final_estimator=LogisticRegression(), n_jobs=4)

## Train

In [17]:
# max_feature_count = [500 * i for i in range(1, 20)]
# f1_scores = []
# for num in max_feature_count:
#     print("max_feature_count:", num)
#     model = Pipeline(
#         [("tfidf2vec", TfidfVectorizer(max_features=num)), ("lr", LogisticRegression())]
#     )
#     model.fit(x_train, y_train)

#     y_pred_class = model.predict(x_train)

#     print(f1_score(y_train, y_pred_class))

#     # print(confusion_matrix(y_train, y_pred_class))
#     # print(classification_report(y_train, y_pred_class))

#     y_pred_class = model.predict(x_dev)
#     val_f1 = f1_score(y_dev, y_pred_class)
#     f1_scores.append(val_f1)
#     # print(confusion_matrix(y_dev, y_pred_class))
#     # print(classification_report(y_dev, y_pred_class))
#     print("*" * 50)

In [16]:
# plt.figure()
# plt.plot(max_feature_count, f1_scores)
# plt.xlabel("TF-IDF features")
# plt.ylabel("F1 Score")
# plt.tight_layout()
# plt.savefig("output/imgs/num_tfidf_features.png")

In [14]:
models = {
    'lr': lr_pipe,
    'nb': nb_pipe,
    'rf': rf_pipe,
    'xgb': xgb_pipe,
    'ada': ada_pipe,
    'grad': grad_pipe,
    # 'svm': svm_pipe,
    'cat': cat_pipe,
    'lgb': lgb_pipe,
    'stacking-nb-rf-xgb': stacking_classifier
}

In [15]:
import joblib

# save
for name, model in tqdm(models.items()):
    model.fit(x_train, y_train)
    joblib.dump(model, f"model-{name}.pkl") 


Learning rate set to 0.073294
0:	learn: 0.6650523	total: 652ms	remaining: 10m 51s
1:	learn: 0.6416700	total: 1.31s	remaining: 10m 52s
2:	learn: 0.6223249	total: 1.97s	remaining: 10m 54s
3:	learn: 0.6067151	total: 2.42s	remaining: 10m 3s
4:	learn: 0.5919185	total: 3.01s	remaining: 9m 58s
5:	learn: 0.5792347	total: 3.56s	remaining: 9m 50s
6:	learn: 0.5681687	total: 4.05s	remaining: 9m 34s
7:	learn: 0.5587038	total: 4.56s	remaining: 9m 25s
8:	learn: 0.5506926	total: 5.13s	remaining: 9m 25s
9:	learn: 0.5422973	total: 5.63s	remaining: 9m 17s
10:	learn: 0.5354553	total: 6.09s	remaining: 9m 7s
11:	learn: 0.5294311	total: 6.53s	remaining: 8m 57s
12:	learn: 0.5239703	total: 6.99s	remaining: 8m 50s
13:	learn: 0.5183675	total: 7.57s	remaining: 8m 53s
14:	learn: 0.5134765	total: 7.95s	remaining: 8m 42s
15:	learn: 0.5088750	total: 8.38s	remaining: 8m 35s
16:	learn: 0.5033530	total: 8.8s	remaining: 8m 28s
17:	learn: 0.4992038	total: 9.31s	remaining: 8m 28s
18:	learn: 0.4951789	total: 9.71s	remaining

In [18]:
# import joblib

# # save
# joblib.dump(model, "model-svm.pkl") 

# # load
# # model = joblib.load("model.pkl")

## Submission

In [19]:
# df_test['label'] = model.predict(df_test['text_clean'])
# submission = df_test[['label']]
# submission.to_csv('./submission.csv')